In [1]:
%load_ext autoreload 
%autoreload 2

import pickle as pkl
import numpy as np
from autodiff.activation import ReLU, Linear
from autodiff.network import Network, NetworkParams

file_path = 'test-parameters.pkl'

with open(file_path, 'rb') as file:
    data = pkl.load(file)

In [2]:
network_def: NetworkParams = {
    "input_shape": 2,
    "output_shape": 1,
    "layers": [
        {
            "input_shape": 2,
            "n_neurons": len(data['w1']),
            "weight_init": data['w1'],
            "bias_init": data['b1'] ,
            "activation": ReLU(),
        },
        {
            "input_shape": len(data['w1']),
            "n_neurons": len(data['w2']),
            "weight_init": data['w2'],
            "bias_init": data['b2'] ,
            "activation": ReLU(),
        },
        {
            "input_shape": len(data['w2']),
            "n_neurons": len(data['w3']),
            "weight_init": data['w3'],
            "bias_init": data['b3'] ,
            "activation": Linear(), 
        }
    ]
} 


In [7]:
import numpy as np
from model import SimpleNet, set_model_weights
import torch
import torch.nn as nn

input_data = data['inputs'][0]
target_data = data['targets'][0]
print(f"{input_data=},  {target_data}")

def get_loss(y, y_hat): 
    return 0.5*((y_hat - y)**2)

# custom network
network = Network(network_def)
y_pred = network.forward(input_data)
y = target_data
loss = get_loss(y, y_pred)
network.backward(y_pred - y) 
print("Custom Backprop Grads")
print("#"*20)
custom_w_gradients = network.layers[0].w_grads
custom_b_gradients = network.layers[0].b_grads
print("First Layer w_grads:   ", custom_w_gradients)
print("First Layer b_grads:   ", custom_b_gradients)
print("#"*20)
criterion = nn.MSELoss()

input_data = torch.tensor(input_data, dtype=torch.float64)
target_data = torch.tensor(target_data, dtype=torch.float64)

# pytorch network
model = SimpleNet()
set_model_weights(data,model)
output = model(input_data)
loss = criterion(output, target_data)/2

loss.backward()

input_data=array([0.09762701, 0.43037873]),  1.0738026310047295
Custom Backprop Grads
####################
First Layer w_grads:    [[-0.0210035  -0.09259178]
 [-0.0184767  -0.08145263]
 [ 0.          0.        ]
 [ 0.01776593  0.0783193 ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [-0.00974717 -0.04296942]
 [ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]]
First Layer b_grads:    [-0.21514022 -0.18925803  0.          0.18197762  0.          0.
 -0.09984094  0.          0.          0.        ]
####################


c:\Users\micha\anaconda3\envs\syde577\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [9]:
# Extract the weight gradients (first element of the tuple)
pytorch_w_gradients = model.get_gradients()[0].detach().cpu().numpy()
pytorch_b_gradients = model.get_gradients()[1].detach().cpu().numpy()


# Comparison function
def compare_gradients(pytorch_gradients: np.ndarray, custom_backprop_gradients: np.ndarray, tolerance: float) -> bool:
    if pytorch_gradients.shape != custom_backprop_gradients.shape:
        raise ValueError('The shapes do not match')

    return np.allclose(pytorch_gradients, custom_backprop_gradients, atol=tolerance)

# Run the comparison
result_w = compare_gradients(
    pytorch_gradients=pytorch_w_gradients,
    custom_backprop_gradients=custom_w_gradients,
    tolerance=0.001
)

print(f"Gradients match W: {result_w}")

result_b = compare_gradients(
    pytorch_gradients=pytorch_b_gradients,
    custom_backprop_gradients=custom_b_gradients,
    tolerance=0.001
)

print(f"Gradients match B: {result_b}")


fc1_weight_grad: tensor([[-0.0210, -0.0926],
        [-0.0185, -0.0815],
        [ 0.0000,  0.0000],
        [ 0.0178,  0.0783],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [-0.0097, -0.0430],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000]], dtype=torch.float64)
fc1_bias_grad: tensor([-0.2151, -0.1893,  0.0000,  0.1820,  0.0000,  0.0000, -0.0998,  0.0000,
         0.0000,  0.0000], dtype=torch.float64)
fc1_weight_grad: tensor([[-0.0210, -0.0926],
        [-0.0185, -0.0815],
        [ 0.0000,  0.0000],
        [ 0.0178,  0.0783],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [-0.0097, -0.0430],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000],
        [ 0.0000,  0.0000]], dtype=torch.float64)
fc1_bias_grad: tensor([-0.2151, -0.1893,  0.0000,  0.1820,  0.0000,  0.0000, -0.0998,  0.0000,
         0.0000,  0.0000], dtype=torch.float64)
Gradients match W: True
Gradients match B: True
